In [1]:
# v2
def check_ownership2(URL,contract,abi): # all inputs are strings
    #connect to the contract
    web3 = Web3(Web3.HTTPProvider(URL))
    web3.eth.defaultAccount = web3.eth.accounts[0]
    abi = json.loads(abi)
    address = web3.toChecksumAddress(contract)
    contract = web3.eth.contract(address = address, abi = abi)
    registry = pd.read_csv('LandRegistry.csv')
    
    #retreive info
    customer_name = contract.functions.customer_name().call()
    property_address = contract.functions.property_address().call()
    #payout_amt = contract.functions.property_address().call()
    #funds_address = web3.toChecksumAddress("contract.functions.us().call()")
    #customer_address = web3.toChecksumAddress("contract.functions.customer().call()")
    
    #modify format of indirizzo to have just the address plus civic number
    split = registry['indirizzo'].str.split(',',expand=True)
    split = split.fillna('')
    split[2] = split[2] + split[3]
    split = split.drop(3,axis=1)
    registry[['Città','Zona','indirizzo']] = split
    
    #find the entry for address in registry
    #and compare owner entry with name to see if they match
    prop = registry.loc[registry.indirizzo == address]

    #if they don't match call the pay_out function
    if prop.Owner.values != name:
        # uncomment the line belowe if we want to keep the funds in another address; as it is now it sends the funds from 
        # private address to the contract and then to the client through the pay_out function, so we can keep our address private
        # if we don't care about that we can directly transfer from our private address to the client's address
        #web3.eth.send_Transaction({'from':funds_address ,'to':address, 'value':web3.toWei(payout_amt, "ether")});
        
        # this calls the pay_out function on the contract, which assumes that the funds are already in the contract;
        contract.functions.pay_out().transact() 


In [2]:
#function for that loops check_onwership every 7 days 
#assuming we have a list containing the address and abi for each insurance contract
import threading

l = None #import the list here

t = None



def loop():
    global t
    for address, abi in l:
        check_ownership(URL,address,abi) #assuming the URL is static; this is the case for ganache, need to change in case it isn't when deloyed
    t = threading.Timer(86400, sayHello)
    t.start()
    
    
# to stop the loop just call t.cancel()